In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install huggingface_hub
!pip install transformers
!pip install datasets

In [ ]:
import os
import pandas as pd
import numpy as np
from datasets import load_dataset
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [33]:
hf_token = os.getenv("llm_project_token")

from datasets import load_dataset
ds = load_dataset("stanfordnlp/imdb", token=hf_token)

ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])
ds_unsupervised = pd.DataFrame(ds['unsupervised'])

In [34]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [35]:
ds['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [36]:
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [37]:
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [38]:
from datasets import Dataset, DatasetDict
# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [39]:
#Checking for nulls
ds_train.isnull().sum()

text     0
label    0
dtype: int64

In [40]:
ds_test.isnull().sum()

text     0
label    0
dtype: int64

In [41]:
stop_w = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words_t = word_tokenize(text)
    words_t = [lemmatizer.lemmatize(word) for word in words_t if word.lower() not in stop_w]
    return ' '.join(words_t)

In [43]:
ds_train['clean_text'] = ds_train['text'].apply(clean_text)
ds_test['clean_text'] = ds_test['text'].apply(clean_text)
ds_unsupervised['clean_text'] = ds_unsupervised['text'].apply(clean_text)

In [45]:
ds_train.to_csv('/content/drive/MyDrive/Colab Notebooks/ds_train.csv', index=False)
ds_test.to_csv('/content/drive/MyDrive/Colab Notebooks/ds_test.csv', index=False)